# Compute Rosseland Mean Opacity with `dsharp_opac`

In [ ]:
import dsharp_opac as opacity
import numpy as np
import matplotlib.pyplot as plt

## Calculate mean opacities for all temperatures

For the Rosseland Mean, we need the *extinction opacity*

$$
\kappa_\nu^\mathrm{ext}(a) = \kappa_\nu^{abs}(a) + (1-g)\,\kappa_\nu^{sca}(a)
$$

then the total opacity of the size distribution is given by

$$
\kappa_\nu^\mathrm{abs,tot} = \frac{\int_0^{a_\mathrm{max}} n(a)\,m\,\kappa_\nu^\mathrm{abs}(a) \,\mathrm{d}a}{\int_0^{a_\mathrm{max}} n(a)\,m \,\mathrm{d}a}
$$

The Planck mean opacity is 

$$
\bar \kappa_\mathrm{P}(T) = \frac{\int_0^\infty \kappa_\nu^\mathrm{abs,tot}\, B_\nu(T)\,\mathrm{d}\nu}{\int_0^\infty  B_\nu(T)\,\mathrm{d}\nu}
$$

and the Rosseland mean opacity is

$$
\bar \kappa_\mathrm{R}(T) = \left( \frac{\int_0^\infty \frac{1}{\kappa_\nu^\mathrm{ext,tot}} \, \frac{\mathrm{d}B_\nu(T)}{\mathrm{d}T}\,\mathrm{d}\nu}{\int_0^\infty  \frac{\mathrm{d}B_\nu(T)}{\mathrm{d}T}\,\mathrm{d}\nu}\right)^{-1}
$$

In [ ]:
with np.load(opacity.get_datafile('default_opacities_smooth.npz')) as d:
    a      = d['a']
    g      = d['g']
    lam    = d['lam']
    k_abs  = d['k_abs']
    k_sca  = d['k_sca']

In [ ]:
def planck_dBnu_dT(freq, T):
    """This function computes the temperature derivative of the blackbody function

            dB_nu(T)        2 h^2 nu^4      exp(h nu / kT)        1
            --------   =    ---------- ------------------------  ---
               dT            k c^2    [ exp(h nu / kT) - 1 ]^2  T^2

    Arguments
    ---------

    freq : float or array
        frequency [in Hz or with astropy.units]

    temp : float or array
        temperature [in K or with astropy.units]

    Returns:
    --------
    dBdT: the temperature derivative of the planck spectrum
        units are using astropy.units if the input values use those, otherwise
        cgs units: erg/(K*s*sr*cm**2*Hz)

    """
    if isinstance(T, u.quantity.Quantity):
        use_units = True
    else:
        T = T * u.K
        use_units = False

    if not isinstance(freq, u.quantity.Quantity):
        freq = freq * u.Hz

    T = np.array(T.value, ndmin=1) * T.unit
    freq = np.array(freq.value, ndmin=1) * freq.unit

    f_ov_T = freq[np.newaxis, :] / T[:, np.newaxis]
    mx = np.floor(np.log(np.finfo(f_ov_T.ravel()[0].value).max))
    exp = np.minimum(f_ov_T * c.h / c.k_B, mx)
    exp = np.maximum(exp, -mx)
    exp = np.exp(exp)

    dBdT = 2 * c.h**2 * freq[np.newaxis, :]**4 / (c.k_B * c.c**2 * T[:, np.newaxis]**2) * \
        1. / (exp * (1. - 1. / exp)**2) / u.sr

    cgsunit = 'erg/(s*Hz*sr*cm**2*K)'
    if use_units:
        return dBdT.to(cgsunit).squeeze()
    else:
        return dBdT.to(cgsunit).value.squeeze()

**here you should interpolate onto your dustpy size grid**

In [ ]:
# define temperature and wavelength arrays

T_array = np.logspace(0, np.log10(1500), 250)
nu = c.c.cgs.value / lam

# calculate the planck function and their derivatives for all those temperatures and wavelength

dBnudT = np.array([planck_dBnu_dT(nu, T) for T in T_array])

In [ ]:
k_ext = k_abs + (1 - g) * k_sca

**here we use a power-law, replace it with dustpy size distribution**
make sure to normalize it

In [ ]:
q = 3.5
amax = 0.1
power_law = a**(4 - q)
power_law[a > amax] = 0
power_law = power_law / power_law.sum()

In [ ]:
k_R_p  = np.zeros_like(T_array)

# for each temperature ...

for it, T in enumerate(T_array):
    
    # sum the absorption opacity

    k_abs_p = (k_abs * power_law[:, None]).sum(0)

    # sum the EXTINCTION opacity

    k_ext_p = (k_ext * power_law[:, None]).sum(0)
    
    # calculate Rosseland opacity for the fit and the power-law
    
    B   = np.trapz(dBnudT[it, :], x=nu)
    k_R_p[it]  = B / np.trapz(dBnudT[it, :].T / k_ext_p, x=nu)

In [ ]:
f, ax = plt.subplots()
ax.loglog(T_array, k_R_p)
ax.set_xlabel('T [K]')
ax.set_xlabel('$\kappa_R$ [cm$^2$/g]');